# 1. Prepare a lot of files

In [1]:
path = 'AA/'
files = ["wiki_0"+str(i) for i in range(9)] + ["wiki_"+str(i) for i in range(10, 14)]

In [2]:
def prep():
    count = 0
    for file in files:
        with open(path+file) as fp: 
            while True:
                markers=0
                with open('done/page'+str(count)+'.txt', 'a') as the_file:
                    while True:
                        line = fp.readline() 

                        if not line: 
                            break
                        if line[0]=='<':
                            markers+=1
                            if markers==2:
                                break
                            continue
                        the_file.write(line)
                count+=1
                if not line:
                    break
# prep()

# 2. Prepare Bag-Of-Words
###### In our case union of all words
We will use PyEnchant

In [3]:
# pip install pyenchant
import enchant
import re

In [4]:
path = 'done/'
files = ['page'+str(i)+'.txt' for i in range(1401)]
bagOfWords = dict()
dictionary = enchant.Dict("en_US")

In [5]:
def isok(char):
    if char >= 'a' and char <= 'z':
        return True
    if char >= '0' and char <= '9':
        return True
    return False

In [ ]:
count=0
bagOfWords = dict() # slows down really quick, so remember to clear dict
for file in files:
    with open(path+file) as fp:
        for fileline in fp:
            line = re.sub("[^\w]", " ", fileline) 
            for word in line.split():
#                 if not isok(word[0].lower()):
#                     word = word[1:]
#                 for i in range(2):
#                     if len(word)>0 and not isok(word[-1].lower()):
#                         word = word[:-1]
#                 if len(word)==0:
#                     continue
                if dictionary.check(word.lower()) and word.lower() not in bagOfWords:
                    bagOfWords[word.lower()]=count
                    count+=1
                    if count % 1e3 == 0:
                        print(count, word)
#                     bagOfWords[word.lower()]+=1
#                 else:
#                     bagOfWords[word.lower()]=0

1000 encouraged
2000 Impressionism
3000 disputed
4000 needs
5000 struck
6000 frequent
7000 chairmen
8000 skiing
9000 1001
10000 bag
11000 descriptive
12000 Mustard
13000 tails
14000 tolerance
15000 concise
16000 popularize
17000 facilitation
18000 provably


Due to rather slow execution, print to check if everything is in order

# 3. For each document prepare bag-of-words and frequency

In [13]:
count=0
wordBags = [dict() for file in files]
for ix, file in enumerate(files):
    with open(path+file) as fp:
        for fileline in fp:
            line = re.sub("[^\w]", " ", fileline) 
            for word in line.split():
#                 if not isok(word[0].lower()):
#                     word = word[1:]
#                 for i in range(2):
#                     if len(word)>0 and not isok(word[-1].lower()):
#                         word = word[:-1]
#                 if len(word)==0:
#                     continue
                count+=1
                if count % 1e5 == 0:
                    print(count, word)
                if word.lower() not in bagOfWords:
                    continue
                if word.lower() not in wordBags[ix]:
                    wordBags[ix][word.lower()]=1
                else:
                    wordBags[ix][word.lower()]+=1

100000 with
200000 higher
300000 which
400000 government
500000 transition
600000 the
700000 2004
800000 and
900000 sources
1000000 of
1100000 finished
1200000 ease
1300000 to
1400000 each
1500000 all
1600000 used
1700000 especially
1800000 In
1900000 report
2000000 professional
2100000 usually
2200000 default
2300000 species
2400000 plants
2500000 amino
2600000 different
2700000 families
2800000 the
2900000 so
3000000 wholeheartedly
3100000 traditional
3200000 are
3300000 the
3400000 determine
3500000 to
3600000 information
3700000 programmable
3800000 entertainment
3900000 contradictory
4000000 as
4100000 the
4200000 will
4300000 including
4400000 purpose
4500000 not
4600000 also
4700000 of
4800000 sounds
4900000 possible
5000000 importance
5100000 the
5200000 successor
5300000 resources
5400000 water
5500000 moons
5600000 for
5700000 Aryabhata
5800000 Kármán


Print to assure process not hang

# 4. Create sparse term-by-document matrix

In [50]:
from scipy import sparse

In [51]:
A = sparse.lil_matrix((len(bagOfWords), len(files)))
for ix, bag in enumerate(wordBags):
    for word in bag:
        A[bagOfWords[word], ix] = bag[word]

# 5. Calulate Inverse Document Frequency and multiply each matrix element

In [60]:
IDF=0
N=len(files)
nw=0
summed = A.getnnz(axis=1)

In [61]:
print(summed[0])
print(list(bagOfWords.keys())[0])
print()

108
japan



In [ ]:

for word in bagOfWords:
    N=